Mount Drive

In [65]:

from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install catboost

     |████████████████████████████████| 76.3 MB 20 kB/s 


Cat-Boost Meta-Learner 

In [66]:
import time
import datetime
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import pandas
import sklearn.metrics as metrics
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from numpy import asarray, hstack, vstack
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold
import pandas as pd
import keras 
import tensorflow
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import catboost as cb
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error

pd.options.mode.chained_assignment = None  
bwd_feat = pd.read_csv('./drive/MyDrive/BorutaShap/bwd_feat.csv')['Feat']
bor_feat = pd.read_csv('./drive/MyDrive/BorutaShap/borutashap.csv')['feat']
knn_feat = pd.read_csv('./drive/MyDrive/BorutaShap/knn_step.csv')['feat']
step_feat = pd.read_csv('./drive/MyDrive/BorutaShap/step_feat.csv')['Feat']
get_bwd_data = FunctionTransformer(lambda x: x[bwd_feat.values], validate=False)
get_bor_data = FunctionTransformer(lambda x: x[bor_feat.values], validate=False)
get_knn_data = FunctionTransformer(lambda x: x[knn_feat.values], validate=False)



def gameweek_rmses(df):
    results = pd.DataFrame(index = np.arange(1,39))
    for name, model in get_models(cat_features=None):
        RMSE = np.sqrt(mean_squared_error(df['total_points'], df[name]))
        results[f'{name}'] = RMSE
    results['Meta'] =  np.sqrt(mean_squared_error(df['total_points'], df['Meta']))
    results.drop_duplicates(inplace = True)
    results['Best'] = results.idxmin(axis=1)
    results['BestRSME'] = results.min(axis=1)
    return results.round(3)

def split_dfs(df, GW, szn = 2020, idx = False):
    X_test = df[(df['season'] == szn) & (df['GW'] == GW)]
    y_test = X_test['total_points']
    X_train = df[df['kickoff_time'] < X_test['kickoff_time'].min()]
    y_train = X_train['total_points']
    if idx == True:
        idxrs_test = X_test[['player_name', 'kickoff_time', 'GW', 'season']]
        X_test.drop(DROP_COLS, axis = 1, inplace = True), X_train.drop(DROP_COLS, axis = 1, inplace = True) 
        return X_train, X_test, y_train, y_test, idxrs_test
    else:
        # Removes kickoff time, gameweek, season and total points
        X_test.drop(DROP_COLS, axis = 1, inplace = True), X_train.drop(DROP_COLS, axis = 1, inplace = True) 
        return X_train, X_test, y_train, y_test


def get_training_data(GW, df, df_encoded, X_oof_train, std_x, std_y): 
    if GW == 1:
        # Get, sort and round numerical precision..
        df =  pd.read_csv('/content/drive/MyDrive/Data/rollbacked_us.csv').round(8).sort_values(by = ['season', 'GW', 'player_name', 'kickoff_time'])
        # X_oof_train = pd.read_csv('./drive/MyDrive/Predictions/oof_predictions_10.csv').round(8)
        X_oof_train = pd.read_csv('./drive/MyDrive/Predictions/unrounded_oof_predictions_10.csv').round(8)
        df = df[df['season'] < 2021] # Predict for /21 
        # Encode cats
        df_encoded = encode_categoricals(df)
        # Scale data
        for data in [df_encoded, df]:
            scaled_cols, error_cols, std_x, std_y = [], [], StandardScaler(), StandardScaler()
            for col in data.select_dtypes(include = 'number').drop([col1 for col1 in DROP_COLS if col1 in data.select_dtypes(include = 'number').columns], axis = 1).columns:
                if data[col].nunique() > 2:
                    scaled_cols.append(col)
                if data[col].min() < 1e-7:
                    error_cols.append(col)
            data[scaled_cols] = std_x.fit_transform(data[scaled_cols])
            data['total_points'] = std_y.fit_transform(data['total_points'].to_numpy().reshape(-1, 1))

        X_oof_train[scaled_cols] = std_x.transform(X_oof_train[scaled_cols])
        X_oof_train.drop(DROP_COLS, axis = 1, inplace = True)
        for name, _ in get_models():
            X_oof_train[name] = std_y.transform(X_oof_train[name].to_numpy().reshape(-1, 1))
    else:
        pass 
      
  
    # Construct data for CBoost base + get indexers
    X_train, X_test, y_train, y_test, idxrs_test = split_dfs(df, GW, idx = True) 

    # Construct data for other base learners
    X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = split_dfs(df_encoded, GW) # Does not contain player name

    return X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, df, df_encoded, std_x, std_y, X_oof_train, idxrs_test

def encode_categoricals(df):
    # Any model that needs numerical preprocessing cannot use the name feature
    encode_df = df.drop(columns = ['player_name'], axis = 1)
    # Encode booleans
    for col in encode_df.columns:
      if pd.api.types.is_bool_dtype(encode_df[col]):
        encode_df[col] = encode_df[col].replace({True:1, False:0})
    # One hot encode 
    encode_df = pd.get_dummies(encode_df, columns=OHE_COLS, prefix=OHE_COLS)
    # Ordinal encoding
    mapping = {'Low': 0, 'Medium': 0.5, 'Hard':1}
    encode_df['FDR'] = encode_df['FDR'].map(lambda x : mapping[x])
    return encode_df

def get_models(cat_features = None):

    models = list()
    # Linear Regression
    models.append(('LR', Pipeline([('selector', get_bwd_data), ('LR', LinearRegression())]))) # Done
    # CatBoost
    models.append(('CBoost', Pipeline([('CBoost', cb.CatBoostRegressor(loss_function='RMSE', cat_features = cat_features,
                                                                       task_type=TASK_TYPE, logging_level='Silent', depth = 6,
                                                                       l2_leaf_reg = 1, learning_rate = 0.03,
                                                                       bagging_temperature = 0.1, iterations = ITERATIONS))]))) 
    # SVR
    models.append(('SVR', Pipeline([('selector', get_bor_data), ('SVR', SVR(C=6, cache_size=200, coef0=0.0, degree=3, epsilon=0.6, gamma=0.005, 
                                                                            kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]))) 
    # KNN
    models.append(('KNN', Pipeline([('selector', get_knn_data), ('KNN', KNeighborsRegressor(algorithm='kd_tree', leaf_size=1, metric='manhattan',
                                                                                            metric_params=None, n_jobs=8, n_neighbors=86, p=2,
                                                                                            weights='distance'))]))) 
    # Neural Network
    models.append(('MLP', Pipeline([('selector', get_bor_data), ('MLP', MLPRegressor(activation='identity', alpha=1e-05, batch_size=16, beta_1=0.9,
                                                                                     beta_2=0.999, early_stopping=True, epsilon=1e-07,
                                                                                     hidden_layer_sizes=(64, 128), learning_rate='constant',
                                                                                     learning_rate_init=0.0001, max_fun=15000, max_iter=50,
                                                                                     momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
                                                                                     power_t=0.5, random_state=0, shuffle=True, solver='adam',
                                                                                     tol=1e-08, validation_fraction=0.1, verbose=False,
                                                                                     warm_start=False))])))  
    return models



def get_out_of_fold_predictions(GW, X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, models):
    meta_X, meta_y, y_index, fold_yhats = list(), list(), list(), list()
    # Store folds
    meta_y.extend(y_test)
    y_index = y_test.index
    # Encode categorical features for base learners' 
    # Get validation predictions of all base learners
    for name, model in models:
        start = time.time()
        if name in ['CBoost']:
            X_train_base, X_test_base = X_train, X_test
            if EARLY_STOPPING:
                model.fit(X_train_base, y_train, CBoost__early_stopping_rounds = 100, CBoost__eval_set = [(X_test_base, y_test)])
            else:
                 model.fit(X_train_base, y_train)
        else:
            X_train_base, X_test_base = X_train_encoded, X_test_encoded
            model.fit(X_train_base, y_train)  

        # Get base learners' predictions
        yhat = model.predict(X_test_base)  
        fold_yhats.append(yhat.reshape(len(yhat), 1)) 
        # Profile model
        if VERBOSE:
            print(f'{GW}: {name}: {round(time.time()-start,3)} seconds') 
    # Store base learner predictions as columns
    meta_X.append(hstack(fold_yhats)) 
    # Scaled out-of-fold predictions of base learners
    y_oof_true = pd.DataFrame(asarray(meta_y), index=y_index)
    X_oof_preds = pd.DataFrame(vstack(meta_X), columns = [name for name, _ in models], index=y_index)
    return X_oof_preds, y_oof_true, y_index 
    

def construct_oof_data(GW, X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, X_oof_train):
    if not PROPAGATE:
      X_train.drop(['team', 'opponent_team', 'player_name'], axis = 1, inplace = True)
      X_test.drop(['team', 'opponent_team', 'player_name'], axis = 1, inplace = True)
    # Categorical features in level-0 training set
    cat_features_train = np.where(X_train.dtypes == 'object')[0] 
    # Base learners 
    models = get_models(cat_features_train) 
    # Get base learners OOF preds
    X_oof_preds, y_oof_true, y_index = get_out_of_fold_predictions(GW, X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, models) 
    # Construct propagated level-1 test set
    X_oof_test = pd.merge(X_oof_preds, X_test, left_index=True, right_index=True) 
    X_oof_test.to_csv(f'/content/drive/MyDrive/Predictions/{GW}_oof.csv', index = False)
    # Categorical features in level-1 testing set
    # Concatenate new data to training data
    if GW > 1:
        X_oof_train = pd.concat([X_oof_train, pd.read_csv(f'/content/drive/MyDrive/Predictions/{GW - 1}_oof.csv')])
    X_oof_train = X_oof_train[X_oof_test.columns]
    return X_oof_test, y_oof_true, cat_features_train,  models, X_oof_train


def fit_meta_model(GW, X_oof_train, y_oof_train, X_oof_test, y_oof_true): 
    start = time.time()
    model =  cb.CatBoostRegressor(cat_features =  np.where(X_oof_train.dtypes == 'object')[0], task_type=TASK_TYPE,
                                  depth = 8, 
                                  iterations = ITERATIONS,
                                  l2_leaf_reg = 1,
                                  learning_rate = 0.03, 
                                  logging_level = 'Silent') # Test: Can early stopping improve performance?
    if EARLY_STOPPING:
        model.fit(X_oof_train, y_oof_train, early_stopping_rounds = 100, eval_set = [(X_oof_test, y_oof_true)])
    else:
        model.fit(X_oof_train, y_oof_train)
    print(f'{GW}: Meta: {round(time.time()-start,3)} seconds') 
    return model

def main():
    # For all gameweeks
    for GW in list(range(1,39)):
        if not VERBOSE:
            print(GW)
        if GW == 1:
            results, df, df_encoded, std_x, std_y, X_oof_train = [], [], [], [], [], []
        else:
            pass
        # Level-0 training + testing sets
        X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, df, df_encoded, std_x, std_y, X_oof_train, idxrs_test = get_training_data(GW, df, df_encoded, X_oof_train, std_x, std_y)
        # Construct level-1 training + testing sets
        X_oof_test, y_oof_true, cat_features_train,  models, X_oof_train = construct_oof_data(GW, X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, X_oof_train)
        # Fit meta-learner on level-1 training set
        meta_model = fit_meta_model(GW, X_oof_train, y_train, X_oof_test, y_oof_true) 
        # Predict using meta_model
        meta_preds = meta_model.predict(X_oof_test)
        # Print weekly RMSE
        if VERBOSE:
            print(f'{GW} : Meta-learner : RMSE = {sqrt(mean_squared_error(np.round(std_y.inverse_transform(y_test)), np.round(std_y.inverse_transform(meta_preds))))}')
            for name, _ in models:
                print(f'{GW} : {name} : RMSE = {sqrt(mean_squared_error(np.round(std_y.inverse_transform(y_test)), np.round(std_y.inverse_transform(X_oof_test[name]))))}')
        # Should response be rounded
        if ROUND:
            for name, _ in models:
                idxrs_test[name] = np.round(std_y.inverse_transform(X_oof_test[name]))
            idxrs_test['Meta'] = np.round(std_y.inverse_transform(meta_preds))
        else: 
            for name, _ in models:
                idxrs_test[name] = std_y.inverse_transform(X_oof_test[name])
            idxrs_test['Meta'] = std_y.inverse_transform(meta_preds)
        # Save weekly results
        results.append(idxrs_test)
    # Concatenate all predictions
    df_preds = pd.concat(results).reset_index(drop=True)    
    # Save the results with the original player name
    df = pd.read_csv('/content/drive/MyDrive/Data/collected_us.csv')[['player_name', 'position', 'team', 'value', 'kickoff_time', 'total_points']]
    df = df[df['kickoff_time'] > '2020-08-12'] # Predicted for the /21 season
    df_preds = pd.merge(df_preds, df, on = ['player_name', 'kickoff_time'])
    df_preds.to_csv('/content/drive/MyDrive/Predictions/new_meta_early_stop_no_rounding.csv', index = False)
    # Use the previous gameweek testing results as early stopping criterion


# Test parameters
N_SPLITS = 10
TASK_TYPE = 'CPU' # GPU is giving inconsistent results
ITERATIONS = 1000 
VERBOSE = True
PROPAGATE = True # Player name, team and opponent team features
ROUND = False
EARLY_STOPPING = True
OHE_COLS =  ['team', 'position', 'opponent_team', 'position_location', 'premium_players']
DROP_COLS = ['kickoff_time', 'total_points', 'GW', 'season']


CatBoost Meta Learner with propagation (> 80 min)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_cboost_vanilla.csv')
#  LR &   CBoost &      SVR &      KNN &      MLP &     Meta &                                               Best &  BestRSME \\

# Total &  741 &       38 &  106.521 &  105.146 &  107.434 &  107.053 &  106.357 &  108.011 &  CBoostLRKNNSVRCBoostCBoostCBoostCBoostMLPLRLRC... &   104.638 \\
# Total &  741 &       38 &  106.954 &  105.731 &  108.202 &  107.678 &  106.838 &  107.520 &  CBoostLRKNNMLPMLPCBoostCBoostCBoostCBoostCBoos... &   104.943 \\
# Total &  741 &       38 &  106.954 &  107.917 &  108.202 &  107.678 &  106.838 &  108.348 &  CBoostLRKNNMLPMLPMLPCBoostKNNMLPLRLRSVRLRLRLRK... &   105.404 \\

# df = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_early_stop_no_rounding.csv')
results = df.groupby('GW').apply(gameweek_rmses).reset_index()
print(results.append(results.sum().rename('Total')).to_latex())
print("Total season performance")
for name in ['LR', 'KNN', 'MLP', 'SVR', 'CBoost', 'Meta']:
    print(f'{name} : {np.sqrt(mean_squared_error(df.total_points, df[name]))}')

\begin{tabular}{lrrrrrrrrlr}
\toprule
{} &   GW &  level\_1 &       LR &   CBoost &      SVR &      KNN &      MLP &     Meta &                                               Best &  BestRSME \\
\midrule
0     &    1 &        1 &    3.356 &    3.122 &    3.172 &    3.312 &    3.169 &    3.362 &                                             CBoost &     3.122 \\
1     &    2 &        1 &    3.505 &    3.633 &    3.653 &    3.611 &    3.554 &    3.838 &                                                 LR &     3.505 \\
2     &    3 &        1 &    2.970 &    3.098 &    2.939 &    2.902 &    2.939 &    2.926 &                                                KNN &     2.902 \\
3     &    4 &        1 &    3.727 &    3.758 &    3.632 &    3.698 &    3.618 &    3.799 &                                                MLP &     3.618 \\
4     &    5 &        1 &    2.635 &    2.772 &    2.779 &    2.653 &    2.628 &    2.786 &                                                MLP &     2.628 \\
5     &

CatBoost Meta Learner with propagation and early stopping (~ 20 min)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_cboost_with_prop_and_early_stop.csv')
results = df.groupby('GW').apply(gameweek_rmses).reset_index()
print(results.append(results.sum().rename('Total')).to_latex())
print("Total season performance")
for name in ['LR', 'KNN', 'MLP', 'SVR', 'CBoost', 'Meta']:
    print(f'{name} : {np.sqrt(mean_squared_error(df.total_points, df[name]))}')

\begin{tabular}{lrrrrrrrrlr}
\toprule
{} &   GW &  level\_1 &       LR &   CBoost &      SVR &      KNN &      MLP &     Meta &                                               Best &  BestRSME \\
\midrule
0     &    1 &        1 &    3.356 &    3.056 &    3.172 &    3.312 &    3.169 &    3.321 &                                             CBoost &     3.056 \\
1     &    2 &        1 &    3.505 &    3.563 &    3.653 &    3.611 &    3.554 &    3.760 &                                                 LR &     3.505 \\
2     &    3 &        1 &    2.970 &    2.928 &    2.939 &    2.902 &    2.939 &    2.906 &                                                KNN &     2.902 \\
3     &    4 &        1 &    3.727 &    3.691 &    3.632 &    3.698 &    3.618 &    3.745 &                                                MLP &     3.618 \\
4     &    5 &        1 &    2.635 &    2.639 &    2.779 &    2.653 &    2.628 &    2.762 &                                                MLP &     2.628 \\
5     &

CatBoost Meta-Learner with prop (wiuth the player name, team, and opponent team features removed) and with early stopping

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_cboost_without_prop_with_early_stopping.csv')
results = df.groupby('GW').apply(gameweek_rmses).reset_index()
print(results.append(results.sum().rename('Total')).to_latex())
print("Total season performance")
for name in ['LR', 'KNN', 'MLP', 'SVR', 'CBoost', 'Meta']:
    print(f'{name} : {np.sqrt(mean_squared_error(df.total_points, df[name]))}')

\begin{tabular}{lrrrrrrrrlr}
\toprule
{} &   GW &  level\_1 &       LR &   CBoost &      SVR &      KNN &      MLP &     Meta &                                               Best &  BestRSME \\
\midrule
0     &    1 &        1 &    3.356 &    3.107 &    3.172 &    3.312 &    3.169 &    3.321 &                                             CBoost &     3.107 \\
1     &    2 &        1 &    3.505 &    3.490 &    3.653 &    3.611 &    3.554 &    3.751 &                                             CBoost &     3.490 \\
2     &    3 &        1 &    2.970 &    2.888 &    2.939 &    2.902 &    2.939 &    2.929 &                                             CBoost &     2.888 \\
3     &    4 &        1 &    3.727 &    3.692 &    3.632 &    3.698 &    3.618 &    3.768 &                                                MLP &     3.618 \\
4     &    5 &        1 &    2.635 &    2.626 &    2.779 &    2.653 &    2.628 &    2.763 &                                             CBoost &     2.626 \\
5     &

Other learners-performance as meta-learners

In [ ]:
def get_meta_models(cat_features = None):

    models = list()
    # Linear Regression
    models.append(('LR', Pipeline([('LR', LinearRegression())]))) 
    # CatBoost
    models.append(('CBoost', Pipeline([('CBoost', cb.CatBoostRegressor(loss_function='RMSE', cat_features = cat_features,
                                                                       task_type=TASK_TYPE, logging_level='Silent', depth = 2,
                                                                       l2_leaf_reg = 3, learning_rate = 0.03,
                                                                       bagging_temperature = 0.1, iterations = ITERATIONS))]))) 
    # SVR
    models.append(('SVR', Pipeline([('SVR', SVR(C=100.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.6, gamma=0.005,
                                               kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])))  
    
    # KNN
    models.append(('KNN', Pipeline([('KNN', KNeighborsRegressor(algorithm='kd_tree', leaf_size=10, metric='euclidean',
                                                                metric_params=None, n_jobs=8, n_neighbors=300, p=2,
                                                                weights='uniform'))]))) 
    # Neural Network
    models.append(('MLP', Pipeline([('MLP', MLPRegressor(activation='relu', alpha=0, batch_size=16, beta_1=0.9,
                                                         beta_2=0.999, early_stopping=True, epsilon=1e-07,
                                                         hidden_layer_sizes=(128, 256), learning_rate='constant',
                                                         learning_rate_init=0.0001, max_fun=15000, max_iter=20,
                                                         momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
                                                         power_t=0.5, random_state=0, shuffle=True, solver='adam',
                                                         tol=1e-08, validation_fraction=0.1, verbose=False,
                                                         warm_start=False))])))  
    return models

def meta_preds(GW, X_oof_train, y_oof_train, X_oof_test, y_oof_test): 
    preds, fold_yhats = list(), list()
    for name, model in get_meta_models():
        start = time.time()
        model.fit(X_oof_train, y_oof_train)
        yhat = model.predict(X_oof_test)  
        fold_yhats.append(yhat.reshape(len(yhat), 1)) 
        print(f'{GW}: Meta: {round(time.time()-start,3)} seconds') 
    preds.append(hstack(fold_yhats)) 
    preds = pd.DataFrame(vstack(preds), columns = [name for name, _ in get_meta_models()])
    return preds

def quick_level_1(GW):  
    # Separate training and testing
    df_train = pd.read_csv('./drive/MyDrive/Predictions/oof_predictions_10.csv')
    df_preds = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_cboost_vanilla.csv')
    std_x, std_y = StandardScaler(), StandardScaler()
    df_concat_train = df_preds[df_preds['GW'] < GW]
    df_test = df_preds[df_preds['GW'] == GW]  
    # Predictors
    X_oof_train  = std_x.fit_transform(pd.concat([df_train[[name for name, _ in get_meta_models()]], 
                                                  df_concat_train[[name for name, _ in get_meta_models()]]]))
    X_oof_test  = std_x.transform(df_test[[name for name, _ in get_meta_models()]])
    
    # Response
    y_oof_train = std_y.fit_transform(pd.concat([df_train['total_points'], 
                                                 df_concat_train['total_points']]).to_numpy().reshape(-1, 1))
    y_oof_test = std_y.fit_transform(df_test['total_points'].to_numpy().reshape(-1, 1))
    # Keep track of players and gameweeks
    test_idx = df_test[['player_name', 'kickoff_time', 'GW', 'season']]
    return X_oof_train, y_oof_train, X_oof_test, y_oof_test, test_idx, std_y

def main():
    # For all gameweeks
    results = []
    for GW in list(range(1,39)):
        # Get level-1 data set
        X_oof_train, y_oof_train, X_oof_test, y_oof_test, idxrs_test, std_y = quick_level_1(GW)
        # Get three models predictions
        meta_predictions = meta_preds(GW, X_oof_train, y_oof_train, X_oof_test, y_oof_test)
        # Save weekly results
        for name, _ in get_meta_models():
            idxrs_test[name] = np.round(std_y.inverse_transform(meta_predictions[name].to_numpy().reshape(-1, 1)))
        # Indexers + results
        idxrs_test['total_points'] = np.round(std_y.inverse_transform(y_oof_test))
        results.append(idxrs_test)
    # Concat all preds
    df_preds = pd.concat(results).reset_index(drop=True)    
    df_preds.to_csv('/content/drive/MyDrive/Predictions/other_metas.csv', index = False)



Simple weighted average of base and meta-learners

In [ ]:
def gameweek_rmses(df):
    results = pd.DataFrame(index = np.arange(1,39))
    for name, model in get_meta_models(cat_features=None):
        RMSE = np.sqrt(mean_squared_error(df['total_points'], df[name]))
        results[f'{name}'] = RMSE
    results['Avg. Base'] =  np.sqrt(mean_squared_error(df['total_points'], df['avg_base_preds']))
    results['Avg. Meta'] =  np.sqrt(mean_squared_error(df['total_points'], df['avg_meta_preds']))
    results.drop_duplicates(inplace = True)
    results['Best'] = results.idxmin(axis=1)
    results['BestRSME'] = results.min(axis=1)
    return results.round(3)


df_base = pd.read_csv('/content/drive/MyDrive/Predictions/new_meta_cboost_vanilla.csv')
df_base['avg_base_preds'] = 0
for name, model in get_models(cat_features=None):
      df_base['avg_base_preds'] += df_base[name]/5


df_preds = pd.read_csv('/content/drive/MyDrive/Predictions/other_metas.csv')
df_preds['avg_meta_preds'] = 0
for name, model in get_meta_models(cat_features=None):
      df_preds['avg_meta_preds'] += df_preds[name]/5

df_preds['avg_meta_preds'] = np.round(df_preds['avg_meta_preds'])
df_preds['avg_base_preds'] = np.round(df_base['avg_base_preds'])

results = df_preds.groupby('GW').apply(gameweek_rmses).reset_index()
print(results.append(results.sum().rename('Total')).to_latex())


\begin{tabular}{lrrrrrrrrrlr}
\toprule
{} &   GW &  level\_1 &       LR &   CBoost &      SVR &      KNN &      MLP &  Average Base &  Average Meta &                                               Best &  BestRSME \\
\midrule
0     &    1 &        1 &    3.111 &    3.059 &    3.130 &    3.112 &    3.106 &         3.124 &         3.078 &                                             CBoost &     3.059 \\
1     &    2 &        1 &    3.592 &    3.591 &    3.642 &    3.572 &    3.628 &         3.562 &         3.572 &                                       Average Base &     3.562 \\
2     &    3 &        1 &    3.063 &    3.040 &    3.024 &    3.011 &    3.057 &         2.930 &         3.049 &                                       Average Base &     2.930 \\
3     &    4 &        1 &    3.776 &    3.740 &    3.772 &    3.719 &    3.832 &         3.660 &         3.727 &                                       Average Base &     3.660 \\
4     &    5 &        1 &    2.684 &    2.698 &    2.773 & 

Remove full stadium matches and test on 2021/22

In [57]:
df = pd.read_csv('./drive/MyDrive/Data/rollbacked_us_testing.csv')
df = df[df['kickoff_time'] > '2020-04-01']
df.groupby(['season'])['GW'].value_counts()
# df[df['season'] == 2021]['kickoff_time'].min()

season  GW
2019    30    362
        38    304
        33    300
        31    299
        35    299
        36    297
        32    296
        37    292
        34    290
2020    26    466
        35    417
        19    413
        24    334
        25    304
        27    302
        30    284
        38    281
        10    279
        7     277
        20    277
        22    277
        31    277
        32    275
        2     274
        3     274
        4     274
        14    274
        15    274
        23    274
        37    274
        5     273
        6     273
        21    272
        28    272
        9     270
        12    270
        13    270
        8     268
        34    247
        11    246
        17    245
        1     220
        16    217
        33    217
        36    214
        18    164
        29    110
2021    5     281
        1     280
        6     275
        2     273
        4     273
        3     270
        7     269
Name: GW, dtype: 

In [ ]:
def construct_level_1_training(X, y, models):
    meta_X, meta_y, y_index = list(), list(), list()
    kfold = KFold(n_splits=N_SPLITS, shuffle=True)
    for k, (train_ix, test_ix) in enumerate(kfold.split(X)):
        fold_yhats = list()
        train_X, test_X = X.iloc[train_ix, :], X.iloc[test_ix, :]
        encoded_train_X, encoded_test_X = encode_categoricals(train_X), encode_categoricals(test_X)
        train_y, test_y = y.iloc[train_ix], y.iloc[test_ix]
        meta_y.extend(test_y)
        y_index.extend(test_ix)
        start = time.time()
         # Out of fold predictions  
        for name, model in models:
            X_train = train_X if name in ['CBoost'] else encoded_train_X
            X_test = test_X if name in ['CBoost'] else encoded_test_X
            model.fit(X_train, train_y)  
            yhat = model.predict(X_test)  
            fold_yhats.append(yhat.reshape(len(yhat), 1)) 
         # Model predictions as columns
        meta_X.append(hstack(fold_yhats))
        print(f'Fold {k+1}: {round(time.time()-start,3)} seconds') 
    # Note, not rounded or unscaled
    meta_X = pd.DataFrame(vstack(meta_X), columns = [name for name, _ in get_models()], index=y_index)
    X_oof_train = pd.merge(meta_X, X, left_index=True, right_index=True) 
    return X_oof_train


def get_models(cat_features = None):

    models = list()
    # Linear Regression
    models.append(('LR', Pipeline([('selector', get_bwd_data), ('LR', LinearRegression())]))) # Done
    # CatBoost
    models.append(('CBoost', Pipeline([('CBoost', cb.CatBoostRegressor(loss_function='RMSE', cat_features = cat_features,
                                                                       task_type=TASK_TYPE, logging_level='Silent', depth = 6,
                                                                       l2_leaf_reg = 1, learning_rate = 0.03,
                                                                       bagging_temperature = 0.1, iterations = ITERATIONS))]))) 
    # SVR
    models.append(('SVR', Pipeline([('selector', get_bor_data), ('SVR', SVR(C=6, cache_size=200, coef0=0.0, degree=3, epsilon=0.6, gamma=0.005, 
                                                                            kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]))) 
    # KNN
    models.append(('KNN', Pipeline([('selector', get_knn_data), ('KNN', KNeighborsRegressor(algorithm='kd_tree', leaf_size=1, metric='manhattan',
                                                                                            metric_params=None, n_jobs=8, n_neighbors=86, p=2,
                                                                                            weights='distance'))]))) 
    # Neural Network
    models.append(('MLP', Pipeline([('selector', get_bor_data), ('MLP', MLPRegressor(activation='identity', alpha=1e-05, batch_size=16, beta_1=0.9,
                                                                                     beta_2=0.999, early_stopping=True, epsilon=1e-07,
                                                                                     hidden_layer_sizes=(64, 128), learning_rate='constant',
                                                                                     learning_rate_init=0.0001, max_fun=15000, max_iter=50,
                                                                                     momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
                                                                                     power_t=0.5, random_state=0, shuffle=True, solver='adam',
                                                                                     tol=1e-08, validation_fraction=0.1, verbose=False,
                                                                                     warm_start=False))])))  
    return models



def get_empty_stadium_training_data(GW, df, df_encoded, X_oof_train, std_x, std_y): 
    if GW == 1:
        # Get, sort and round numerical precision..
        df = pd.read_csv('./drive/MyDrive/Data/rollbacked_us_testing.csv').round(8).sort_values(by = ['season', 'GW', 'player_name', 'kickoff_time'])
        df = df[df['kickoff_time'] > '2020-04-01'] # Empty stadiums
        df.reset_index(inplace = True, drop = True)
        # Encode cats
        df_encoded = encode_categoricals(df)
        # Scale data
        for data in [df_encoded, df]:
            scaled_cols, error_cols, std_x, std_y = [], [], StandardScaler(), StandardScaler()
            for col in data.select_dtypes(include = 'number').drop([col1 for col1 in DROP_COLS if col1 in data.select_dtypes(include = 'number').columns], axis = 1).columns:
                if data[col].nunique() > 2:
                    scaled_cols.append(col)
                if data[col].min() < 1e-7:
                    error_cols.append(col)
            data[scaled_cols] = std_x.fit_transform(data[scaled_cols])
            data['total_points'] = std_y.fit_transform(data['total_points'].to_numpy().reshape(-1, 1))

        # Create a fresh OOF training set for the /22 season
        X, _, y, _ = split_dfs(df, GW, szn = 2021) 
        X_oof_train = construct_level_1_training(X, y, models = get_models(cat_features = np.where(X.dtypes == 'object')[0]))
        X_oof_train.to_csv('/content/drive/MyDrive/Predictions/out_of_fold_21.csv', index = False)
        # X_oof_train =  pd.read_csv('/content/drive/MyDrive/Predictions/out_of_fold_21.csv').round(8).sort_values(by = ['season', 'GW', 'player_name', 'kickoff_time'])
    else:
        pass 

  
    # Construct data for CBoost base + get indexers
    X_train, X_test, y_train, y_test, idxrs_test = split_dfs(df, GW, idx = True, szn = 2021) 

    # Construct data for other base learners
    X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = split_dfs(df_encoded, GW, szn = 2021) # Does not contain player name
    
    return X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, df, df_encoded, std_x, std_y, X_oof_train, idxrs_test

def main():
    # For all gameweeks
    for GW in list(range(1,8)): # Predict for first seven gameweeks
        if not VERBOSE:
            print(GW)
        if GW == 1:
            results, df, df_encoded, std_x, std_y, X_oof_train = [], [], [], [], [], []
        else:
            pass
        # Level-0 training + testing sets
        X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, df, df_encoded, std_x, std_y, X_oof_train, idxrs_test = get_empty_stadium_training_data(GW, df, df_encoded, X_oof_train, std_x, std_y)
        # Construct level-1 training + testing sets
        X_oof_test, y_oof_true, cat_features_train,  models, X_oof_train = construct_oof_data(GW, X_train, X_test, X_train_encoded, X_test_encoded, y_train, y_test, X_oof_train)
        # Fit meta-learner on level-1 training set
        meta_model = fit_meta_model(GW, X_oof_train, y_train, X_oof_test, y_oof_true) 
        # Predict using meta_model
        meta_preds = meta_model.predict(X_oof_test)
        # Print weekly RMSE
        if VERBOSE:
            print(f'{GW} : Meta-learner : RMSE = {sqrt(mean_squared_error(np.round(std_y.inverse_transform(y_test)), np.round(std_y.inverse_transform(meta_preds))))}')
            for name, _ in models:
                print(f'{GW} : {name} : RMSE = {sqrt(mean_squared_error(np.round(std_y.inverse_transform(y_test)), np.round(std_y.inverse_transform(X_oof_test[name]))))}')
        # Should response be rounded
        if ROUND:
            for name, _ in models:
                idxrs_test[name] = np.round(std_y.inverse_transform(X_oof_test[name]))
            idxrs_test['Meta'] = np.round(std_y.inverse_transform(meta_preds))
        else: 
            for name, _ in models:
                idxrs_test[name] = std_y.inverse_transform(X_oof_test[name])
            idxrs_test['Meta'] = std_y.inverse_transform(meta_preds)
        # Save weekly results
        results.append(idxrs_test)
    # Concatenate all predictions
    df_preds = pd.concat(results).reset_index(drop=True)    
    # Save the results with the original player name
    df = pd.read_csv('/content/drive/MyDrive/Data/collected_us_updated.csv')[['player_name', 'position', 'team', 'value', 'kickoff_time', 'total_points']]
    df = df[df['kickoff_time'] > '2021-08-01'] # Predicted for the /22 season
    df_preds = pd.merge(df_preds, df, on = ['player_name', 'kickoff_time'])
    df_preds.to_csv('/content/drive/MyDrive/Predictions/empty_stadium_performance.csv', index = False)
    # Use the previous gameweek testing results as early stopping criterion

# Test parameters
N_SPLITS = 10
TASK_TYPE = 'CPU' # GPU is giving inconsistent results
ITERATIONS = 1000 
VERBOSE = True
PROPAGATE = True # Player name, team and opponent team features
ROUND = True
EARLY_STOPPING = True
OHE_COLS =  ['team', 'position', 'opponent_team', 'position_location', 'premium_players']
DROP_COLS = ['kickoff_time', 'total_points', 'GW', 'season']
main()

In [ ]:
def gameweek_rmses(df):
    results = pd.DataFrame(index = np.arange(1,8))
    for name, model in get_models(cat_features=None):
        RMSE = np.sqrt(mean_squared_error(df['total_points'], df[name]))
        results[f'{name}'] = RMSE
    results['Meta'] =  np.sqrt(mean_squared_error(df['total_points'], df['Meta']))
    results.drop_duplicates(inplace = True)
    results['Best Learner'] = results.idxmin(axis=1)
    results['Best RMsE'] = results.min(axis=1)
    return results.round(3)

df = pd.read_csv('/content/drive/MyDrive/Predictions/empty_stadium_performance.csv')
results = df.groupby('GW').apply(gameweek_rmses).reset_index()
print(results.append(results.sum().rename('Total')).to_latex())
print("Total season performance")
for name in ['LR', 'KNN', 'MLP', 'SVR', 'CBoost', 'Meta']:
    print(f'{name} : {np.sqrt(mean_squared_error(df.total_points, df[name]))}')